In [1]:
QRELS_PATH = 'evaluation/qrels.json'

In [114]:
SEED = 2024
ANALYSIS_COL = 'userAcceleration.z'
PRECOMPUTED_DISTANCES = f"evaluation/ddtw/{ANALYSIS_COL.replace('.', '-')}_distances.npy"
RUN_FILE = f"evaluation/runs/ddtw_{ANALYSIS_COL.replace('.', '-')}.json"

# DIST = 'adtw'
# PRECOMPUTED_DISTANCES = f"evaluation/multivar/{DIST}_distances.npy"
# RUN_FILE = f"evaluation/runs/{DIST}_multivar.json"

# Preparando avaliação

In [115]:
from ranx import Qrels

In [116]:
qrels = Qrels.from_file(QRELS_PATH)

# Executando a busca por similaridade

In [117]:
from ranx import Run
import numpy as np
from aeon.distances import dtw_pairwise_distance

In [118]:
def compute_neighbors_by_similarity(
    distances: np.array,
    topk: int = 100) -> dict:

  run_dict = {}

  for qid in range(distances.shape[0]):
    all_sims = distances[qid]
    topk = np.argsort(all_sims)

    # ignorando o proprio exemplo de busca
    estimated_ids = topk[1:]
    scores = list(range(len(estimated_ids)))[::-1]

    run_dict[str(qid)] = {str(n): float(score) for n, score in zip(estimated_ids, scores)}

  return run_dict

In [119]:
PRECOMPUTED_DISTANCES

'evaluation/ddtw/userAcceleration-z_distances.npy'

In [120]:
distances = np.load(PRECOMPUTED_DISTANCES)

In [121]:
run_dict = compute_neighbors_by_similarity(distances)

# Avaliando métricas de RI

In [122]:
from ranx import evaluate

In [123]:
METRICS = ["recall@10", 'map@10', 'hit_rate@10', 'mrr@10']
# METRICS = 'mrr@10'

In [124]:
run = Run(run_dict)

In [125]:
metrics = evaluate(qrels, run, METRICS)
metrics

{'recall@10': 0.22743055555555555,
 'map@10': 0.17848473783803642,
 'hit_rate@10': 0.9791666666666666,
 'mrr@10': 0.7875771604938272}

# Serializando Run

In [126]:
RUN_FILE

'evaluation/runs/ddtw_userAcceleration-z.json'

In [127]:
run.save(RUN_FILE)